# ReAct Agent in LangGraph

source from LangGraph source-code example

example code, from [LangGraph agents](https://langchain-ai.github.io/langgraph/reference/agents/)
该代码样例也可以在 langgraph 源码里找到

In [2]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     base_url="https://lxxxxx.enovo.com/v1/", 
#     api_key="sxxxxxxxwW",
#     model_name="qwen2.5-instruct"
#     )

llm = ChatOpenAI(
    base_url="https://api.deepseek.com", 
    api_key="sk-3b458ee0624f41e1b8c589e74be23e44",
    model_name="deepseek-chat"
    )

result = llm.invoke('Hello, how are you?')
result.pretty_print()

================================== Ai Message ==================================

Hello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you with anything you need. 😊 How about you? How are you doing today?


LangGraph 函数和参数讲解
```python
create_react_agent(
    model: Union[str, LanguageModelLike],
    tools: Union[
        Sequence[Union[BaseTool, Callable, dict[str, Any]]],
        ToolNode,
    ],
    *,
    prompt: Optional[Prompt] = None,
    response_format: Optional[
        Union[
            StructuredResponseSchema,
            tuple[str, StructuredResponseSchema],
        ]
    ] = None,
    pre_model_hook: Optional[RunnableLike] = None,
    post_model_hook: Optional[RunnableLike] = None,
    state_schema: Optional[StateSchemaType] = None,
    config_schema: Optional[Type[Any]] = None,
    checkpointer: Optional[Checkpointer] = None,
    store: Optional[BaseStore] = None,
    interrupt_before: Optional[list[str]] = None,
    interrupt_after: Optional[list[str]] = None,
    debug: bool = False,
    version: Literal["v1", "v2"] = "v2",
    name: Optional[str] = None
) -> CompiledGraph
```

In [17]:
from langgraph.prebuilt import create_react_agent

def check_weather(location: str) -> str:
    '''Return the weather forecast for the specified location.'''
    return f"It's always sunny in {location}"

system_prompt = "You are a helpful bot named Fred."

graph = create_react_agent(
    model=llm,
    tools=[check_weather],
    prompt=system_prompt
)

inputs = {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
for chunk in graph.stream(inputs, stream_mode="updates"):
    print(chunk)

for chunk in graph.stream(inputs, stream_mode="values"):
    chunk['messages'][-1].pretty_print()

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_b100d67a-172d-4379-9ad9-374461af4455', 'function': {'arguments': '{"location":"sf"}', 'name': 'check_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 112, 'total_tokens': 132, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 48}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '7f86d51a-92ce-4fca-b8f1-a0f2eca7b646', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5411223f-71a7-45d8-9412-a1760eaef892-0', tool_calls=[{'name': 'check_weather', 'args': {'location': 'sf'}, 'id': 'call_0_b100d67a-172d-4379-9ad9-374461af4455', 'type': 'tool_call'}], usage_metadata={'input_tokens': 112, 'output_tokens': 20, 'total_tokens': 132

这里我们可以使用更复杂的 prompt 来让 LLM 更好地理解我们的需求, 下面是两个例子

In [5]:
#### Add a more complex prompt for the LLM:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful bot named Fred."),
    ("placeholder", "{messages}"),
    ("user", "Remember, always be polite!"),
    ])

graph = create_react_agent(llm, tools=[check_weather], prompt=prompt)
inputs = {"messages": [("user", "What's your name? And what's the weather in SF?")]}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What's your name? And what's the weather in SF?
================================== Ai Message ==================================

Of course! My name is Fred, and I'm happy to help. Let me check the weather in San Francisco for you—just a moment!
Tool Calls:
  check_weather (call_0_dfb483ba-311b-46eb-bd05-87eb3a7bdad2)
 Call ID: call_0_dfb483ba-311b-46eb-bd05-87eb3a7bdad2
  Args:
    location: San Francisco
================================= Tool Message =================================
Name: check_weather

It's always sunny in San Francisco
================================== Ai Message ==================================

You're absolutely right—politeness is key!  

**My name is Fred**, and I'm here to assist you.  

As for the weather in **San Francisco**, it seems to be **sunny** today. Let me know if there's anything else I can help you with—I'm happy to oblige! 😊


In [10]:
#### Add complex prompt with custom graph state:
from typing_extensions import TypedDict
from langgraph.managed import IsLastStep
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated

prompt = ChatPromptTemplate.from_messages([
         ("system", "Today is {today}"),
         ("placeholder", "{messages}"),
    ])

class CustomState(TypedDict):
    today: str
    messages: Annotated[list[BaseMessage], add_messages]
    is_last_step: IsLastStep
    remaining_steps: int

graph = create_react_agent(
    model=llm, tools=[check_weather], state_schema=CustomState, prompt=prompt
)
inputs = {"messages": [("user", "What's today's date? And what's the weather in SF?")], "today": "July 16, 2004"}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What's today's date? And what's the weather in SF?
================================== Ai Message ==================================

Today's date is **July 16, 2024**. 

Let me check the weather in San Francisco for you.
Tool Calls:
  check_weather (call_0_d31954bf-62d0-45ef-8d11-cadc16e33582)
 Call ID: call_0_d31954bf-62d0-45ef-8d11-cadc16e33582
  Args:
    location: San Francisco
================================= Tool Message =================================
Name: check_weather

It's always sunny in San Francisco
================================== Ai Message ==================================

The weather in San Francisco today is **sunny**. Enjoy the sunshine!


In [11]:
#### Add thread-level "chat memory" to the graph:

from langgraph.checkpoint.memory import MemorySaver

graph = create_react_agent(model=llm, tools=[check_weather], checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "thread-1"}}

def print_stream(graph, inputs, config):
    for s in graph.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "What's the weather in SF?")]}
print_stream(graph, inputs, config)
inputs2 = {"messages": [("user", "Cool, so then should i go biking today?")]}
print_stream(graph, inputs2, config)



================================ Human Message =================================

What's the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  check_weather (call_0_2b70c815-2b06-4a43-ade3-46797e349ee6)
 Call ID: call_0_2b70c815-2b06-4a43-ade3-46797e349ee6
  Args:
    location: SF
================================= Tool Message =================================
Name: check_weather

It's always sunny in SF
================================== Ai Message ==================================

The weather in SF is currently sunny!
================================ Human Message =================================

Cool, so then should i go biking today?
================================== Ai Message ==================================

Absolutely! Since it's sunny in SF, it's a great day for biking. Just remember to stay hydrated, wear sunscreen, and enjoy the ride! If you're biking near busy areas, stay safe and follow traffic rules. Have 

In [12]:
### Add an interrupt to let the user confirm before taking an action:

graph = create_react_agent(model=llm, tools=[check_weather], checkpointer=MemorySaver())

config = {"configurable": {"thread_id": "thread-1"}}

inputs = {"messages": [("user", "What's the weather in SF?")]}
print_stream(graph, inputs, config)
snapshot = graph.get_state(config)
print("Next step: ", snapshot.next)
print_stream(graph, None, config)

================================ Human Message =================================

What's the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  check_weather (call_0_5f927f21-d96e-4ea2-832f-532996f9d4c5)
 Call ID: call_0_5f927f21-d96e-4ea2-832f-532996f9d4c5
  Args:
    location: SF
================================= Tool Message =================================
Name: check_weather

It's always sunny in SF
================================== Ai Message ==================================

The weather in SF is always sunny! Enjoy the beautiful day!
Next step:  ()
================================== Ai Message ==================================

The weather in SF is always sunny! Enjoy the beautiful day!


In [ ]:
### Add cross-thread memory and timeout to the graph:

from typing import Annotated, Sequence
from langgraph.prebuilt import InjectedStore
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig

def save_memory(
        memory: str, *, 
        config: RunnableConfig, 
        store: Annotated[BaseStore, InjectedStore()]
) -> str:
    '''Save the given memory for the current user.'''
    # This is a **tool** the model can use to save memories to storage
    user_id = config.get("configurable", {}).get("user_id")
    namespace = ("memories", user_id)
    store.put(namespace, f"memory_{len(store.search(namespace))}", {"data": memory})
    return f"Saved memory: {memory}"

class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

def prepare_model_inputs(
        state: AgentState, 
        config: RunnableConfig, 
        store: BaseStore
):
    # Retrieve user memories and add them to the system message
    # This function is called **every time** the model is prompted. It converts the state to a prompt
    user_id = config.get("configurable", {}).get("user_id")
    namespace = ("memories", user_id)
    memories = [m.value["data"] for m in store.search(namespace)]
    system_msg = f"User memories: {', '.join(memories)}"
    return [{"role": "system", "content": system_msg}] + state["messages"]

from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
store = InMemoryStore()
graph = create_react_agent(
    model=llm, tools=[save_memory, check_weather], 
    prompt=prepare_model_inputs, 
    store=store, checkpointer=MemorySaver()
)
config = {"configurable": {"thread_id": "thread-1", "user_id": "1"}}
inputs = {"messages": [("user", "Hey I'm Will, how's it going?")]}
print_stream(graph, inputs, config)
inputs2 = {"messages": [("user", "I like to bike, the weather is good to bike?")]}
print_stream(graph, inputs2, config)

config = {"configurable": {"thread_id": "thread-2", "user_id": "1"}}
inputs3 = {"messages": [("user", "Hi！I go to bike but it rains here!")]}
print_stream(graph, inputs3, config)

import time



================================ Human Message =================================

Hey I'm Will, how's it going?
================================== Ai Message ==================================

Hey Will! I'm just a virtual assistant, so I'm always here and ready to help. How's it going with you? Anything on your mind or need assistance with?
================================ Human Message =================================

I like to bike, the weather is good to bike?
================================== Ai Message ==================================
Tool Calls:
  check_weather (call_0_54cdb21e-1652-494c-912f-1c4e42c5f5e3)
 Call ID: call_0_54cdb21e-1652-494c-912f-1c4e42c5f5e3
  Args:
    location: current
================================= Tool Message =================================
Name: check_weather

It's always sunny in current
================================== Ai Message ==================================

The weather is sunny where you are, so it sounds like a perfect day for bikin

A feasible design for a React agent in a system as follows:

In [ ]:


from typing import Annotated, Sequence, TypedDict

from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import create_react_agent

model = ChatOpenAI(
    base_url="https://api.deepseek.com", 
    api_key="sk-3b458ee0624f41e1b8c589e74be23e44",
    model_name="deepseek-chat"
    )

class State(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]
    current_idx: int
    current_task: str




def print_stream(graph, inputs, config):
    for s in graph.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "What's the weather in SF?")]}

